In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/Shareddrives/M G/AI/NSEI.csv")
print(data)

           Date          Open          High           Low         Close  \
0    2021-04-26  14449.450195  14557.500000  14421.299805  14485.000000   
1    2021-04-27  14493.799805  14667.549805  14484.849609  14653.049805   
2    2021-04-28  14710.500000  14890.250000  14694.950195  14864.549805   
3    2021-04-29  14979.000000  15044.349609  14814.450195  14894.900391   
4    2021-04-30  14747.349609  14855.450195  14601.700195  14631.099609   
..          ...           ...           ...           ...           ...   
243  2022-04-18  17183.449219  17237.750000  17067.849609  17173.650391   
244  2022-04-19  17258.949219  17275.650391  16824.699219  16958.650391   
245  2022-04-20  17045.250000  17186.900391  16978.949219  17136.550781   
246  2022-04-21  17234.599609  17414.699219  17215.500000  17392.599609   
247  2022-04-22  17242.750000  17315.300781  17149.199219  17171.949219   

        Adj Close  Volume  
0    14485.000000  452700  
1    14653.049805  451800  
2    14864.5498

# Agent

In [ ]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import numpy as np
import random
from collections import deque

class Agent:
	def __init__(self, state_size, is_eval=False, model_name=""):
		self.state_size = state_size # normalized previous days
		self.action_size = 3 # sit, buy, sell
		self.memory = deque(maxlen=1000)
		self.inventory = []
		# self.model_name = "model_ep2"
		self.is_eval = is_eval

		self.gamma = 0.95
		self.epsilon = 1.0
		self.epsilon_min = 0.01
		self.epsilon_decay = 0.995

		self.model = load_model("/content/drive/Shareddrives/M G/AI/model_ep7") if is_eval else self._model()

	def _model(self):
		model = Sequential()
		model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
		model.add(Dense(units=32, activation="relu"))
		model.add(Dense(units=8, activation="relu"))	
		model.add(Dense(self.action_size, activation="linear"))
		model.compile(loss="mse", optimizer=Adam(lr=0.001)) 

		return model

	def act(self, state): # predict the next action to be taken (sell, buy)
		if not self.is_eval and random.random() <= self.epsilon:
			return random.randrange(self.action_size)

		options = self.model.predict(state)
		return np.argmax(options[0])

	def expReplay(self, batch_size): # if memory gets full it will reset the memory
		mini_batch = []
		l = len(self.memory)
		for i in range(l - batch_size + 1, l):
			mini_batch.append(self.memory[i])

		for state, action, reward, next_state, done in mini_batch:
			target = reward
			if not done:
				target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

			target_f = self.model.predict(state)
			target_f[0][action] = target
			self.model.fit(state, target_f, epochs=1, verbose=0)

		if self.epsilon > self.epsilon_min:
			self.epsilon *= self.epsilon_decay 

In [ ]:
import numpy as np
import math

# prints formatted price
def formatPrice(n):
	return ("-$" if n < 0 else "$") + "{0:.2f}".format(abs(n))

# returns the vector containing stock data from a fixed file
def getStockDataVec():
	vec = []
	lines = open("/content/drive/Shareddrives/M G/AI/NSEI.csv", "r").read().splitlines()

	for line in lines[1:]:
		vec.append(float(line.split(",")[4]))

	return vec

# returns the sigmoid



def sigmoid(x):
	try:
		return 1 / (1 + math.exp(-x))
	except OverflowError:
		ans = float('inf')
		return 0

# returns an an n-day state representation ending at time t
def getState(data, t, n):
	d = t - n + 1
	block = data[d:t + 1] if d >= 0 else -d * [data[0]] + data[0:t + 1] # predicting the model on consecutive days difference
	res = []
	for i in range(n - 1):
		res.append(sigmoid(block[i + 1] - block[i]))

	return np.array([res])

In [ ]:
# from agent.agent import Agent
# from functions import *
import sys

if len(sys.argv) != 4:
	print("Usage: python train.py [stock] [window] [episodes]")
	exit()


window_size = 10 # 10 days stocks to count the difference
episode_count = 10
agent = Agent(window_size)
data = getStockDataVec()
l = len(data) - 1
batch_size = 32 #

for e in range(episode_count + 1):
	print("Episode " + str(e) + "/" + str(episode_count))
	state = getState(data, 0, window_size + 1)

	total_profit = 0
	agent.inventory = []

	for t in range(l):
		action = agent.act(state)

		# sit
		next_state = getState(data, t + 1, window_size + 1)
		reward = 0

		if action == 1: # buy
			agent.inventory.append(data[t])
			print("Buy: " + formatPrice(data[t]))

		elif action == 2 and len(agent.inventory) > 0: # sell
			bought_price = agent.inventory.pop(0)
			reward = max(data[t] - bought_price, 0)
			total_profit += data[t] - bought_price
			print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

		done = True if t == l - 1 else False
		agent.memory.append((state, action, reward, next_state, done))
		state = next_state

		if done:
			print("--------------------------------")
			print("Total Profit: " + formatPrice(total_profit))
			print("--------------------------------")

		if len(agent.memory) > batch_size:
			agent.expReplay(batch_size)

	if e % 2 == 0:
		agent.model.save("/content/drive/Shareddrives/M G/AI" + str(e))

In [ ]:
import keras
from keras.models import load_model

# from agent.agent import Agent
# from functions import *
import sys

if len(sys.argv) != 3:
	print("Usage: python evaluate.py [stock] [model]")
	exit()
opy the content of file 'file.txt' to file 'file2.txt'
model = load_model("/content/drive/Shareddrives/M G/AI/model_ep7")
window_size = model.layers[0].input.shape.as_list()[1]

agent = Agent(window_size, True)
data = getStockDataVec()
l = len(data) - 1
batch_size = 32

state = getState(data, 0, window_size + 1)
# print(state)

total_profit = 0
agent.inventory = []


for t in range(l):
	action = agent.act(state)
	# print(action)
	# sit
	next_state = getState(data, t + 1, window_size + 1)
	reward = 0

	if action == 1: # buy
		agent.inventory.append(data[t])
		print("Buy: " + formatPrice(data[t]))

	elif action == 2 and len(agent.inventory) > 0: # sell
		bought_price = agent.inventory.pop(0)
		reward = max(data[t] - bought_price, 0)
		total_profit += data[t] - bought_price
		print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))

	done = True if t == l - 1 else False
	agent.memory.append((state, action, reward, next_state, done))
	state = next_state

	if done:
		print("--------------------------------")
		print( " Total Profit: " + formatPrice(total_profit))
		print("--------------------------------")

Buy: $17101.95
Buy: $17339.85
Sell: $17576.85 | Profit: $474.90
Sell: $17780.00 | Profit: $440.15
Buy: $16247.95
Buy: $16658.40
Sell: $16793.90 | Profit: $545.95
Buy: $16498.05
Buy: $16245.35
Buy: $15863.15
Sell: $16013.45 | Profit: -$644.95
Sell: $16345.35 | Profit: -$152.70
Sell: $16630.45 | Profit: $385.10
Sell: $16871.30 | Profit: $1008.15
--------------------------------
 Total Profit: $2056.60
--------------------------------


In [ ]:
# Buy: $3476.40
# Buy: $3484.20
# Sell: $3492.25 | Profit: $15.85
# Sell: $3508.65 | Profit: $24.45
# --------------------------------
#  Total Profit: $40.30
# -------------------------------

# model ep 7
# Buy: $17101.95
# Buy: $17339.85
# Sell: $17576.85 | Profit: $474.90
# Sell: $17780.00 | Profit: $440.15
# Buy: $16247.95
# Buy: $16658.40
# Sell: $16793.90 | Profit: $545.95
# Buy: $16498.05
# Buy: $16245.35
# Buy: $15863.15
# Sell: $16013.45 | Profit: -$644.95
# Sell: $16345.35 | Profit: -$152.70
# Sell: $16630.45 | Profit: $385.10
# Sell: $16871.30 | Profit: $1008.15
# --------------------------------
#  Total Profit: $2056.60
# --------------------------------